##### source code: https://colab.research.google.com/drive/1ysEKrw_LE2jMndo1snrZUh5w87LQsCxk#forceEdit=true&sandboxMode=true&scrollTo=CRGOx6_v4eZ_

In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 1us/step


In [4]:
# from google.colab import files      # to upload file in google colab env
# path_to_file = list(files.upload().keys())[0]

In [5]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [10]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [12]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [13]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [14]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [17]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 3.28550814e-03 -3.21296416e-03 -3.94526054e-04 ...  1.78730488e-03
   -2.02260958e-03  7.91079458e-03]
  [ 4.07655584e-03  4.01577400e-03 -6.31302129e-03 ...  6.36673532e-03
    2.60268163e-04  6.78106630e-03]
  [ 9.03888512e-03 -2.90617370e-03 -6.54605823e-03 ...  5.64712193e-03
   -2.69289571e-03  4.70679067e-03]
  ...
  [ 1.37190213e-02  9.66157066e-04 -4.80533252e-03 ... -1.36888318e-03
    3.67474742e-03  2.63784284e-04]
  [ 1.05617484e-02  7.75970565e-03 -4.25249990e-03 ... -3.46542895e-03
    1.73613266e-03 -1.88962277e-03]
  [ 7.65290158e-03  5.95134450e-03 -7.81845418e-04 ... -6.79687969e-03
    2.35800818e-03 -6.29253639e-03]]

 [[-5.64363872e-05  3.57312406e-03  1.78367272e-03 ...  5.10888174e-03
    3.41656269e-03 -8.88154435e-04]
  [ 4.78611281e-03  3.35812429e-03  1.23798521e-03 ... -7.61700142e-03
    2.03779782e-03 -1.18981604e-03]
  [ 3.32397129e-03  3.99772078e-03  3.21600237e-03 ... -9.03803203e-03
    2.62341718e-03 -6.06141612e-03]
  ...
  [-5.591

In [18]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00328551 -0.00321296 -0.00039453 ...  0.0017873  -0.00202261
   0.00791079]
 [ 0.00407656  0.00401577 -0.00631302 ...  0.00636674  0.00026027
   0.00678107]
 [ 0.00903889 -0.00290617 -0.00654606 ...  0.00564712 -0.0026929
   0.00470679]
 ...
 [ 0.01371902  0.00096616 -0.00480533 ... -0.00136888  0.00367475
   0.00026378]
 [ 0.01056175  0.00775971 -0.0042525  ... -0.00346543  0.00173613
  -0.00188962]
 [ 0.0076529   0.00595134 -0.00078185 ... -0.00679688  0.00235801
  -0.00629254]], shape=(100, 65), dtype=float32)


In [19]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 3.2855081e-03 -3.2129642e-03 -3.9452605e-04  8.9770707e-04
  4.6463474e-03  6.6446164e-04  4.2595898e-04 -1.2319670e-03
  3.9452389e-03  3.1783676e-03 -2.2988691e-04 -7.9560844e-04
 -3.0262363e-03 -6.0070731e-04 -1.0714831e-03 -1.5464957e-03
 -1.7502138e-03 -2.3682383e-03  1.5039917e-03  1.3325666e-03
 -7.5481427e-03 -3.5766896e-03  5.2947900e-04  7.3241475e-03
  1.4551028e-05  4.1994629e-03 -7.4335630e-04  1.0522607e-03
 -2.7264988e-03  9.7377750e-05  3.0652946e-03  1.4019742e-03
 -1.5395524e-03  2.6159799e-03 -7.6197169e-04  2.7099662e-03
  1.5844451e-03  2.3364858e-03 -3.3320258e-03 -8.5290126e-04
 -3.8816482e-03 -1.7563921e-03 -1.9367144e-04  3.2742070e-03
 -6.6873961e-04 -1.0884928e-03 -7.7934528e-04 -4.2897416e-03
 -1.6705997e-04  1.8924256e-03  1.1772482e-03  3.7230311e-03
 -3.9775576e-03 -5.8540492e-03  2.6282419e-03  5.6976737e-03
  1.5590955e-03  5.5066561e-03  2.4569922e-03  7.3113461e-04
 -6.6230638e-04 -4.5616622e-04  1.7873049e-03 -2.0226096e-03
  7.910794

In [20]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"PEgORQerjUxFw-;HJrzqyUyEghQucSjINj\nLfJg$'I:GCBCxhqbrovIT\nHKjLTIVDKOpn,THGaBJj;guS:ttXqsLiGekg&MKNC.;"

In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [22]:
model.compile(optimizer='adam', loss=loss)

In [23]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))